Importando Librerías

In [1]:
#!/usr/bin/python
import random
import numpy as np

Tempo: A más actividad más tempo

In [159]:
#Selección Tempo 40bmp -180bpm
def selector_tempo(act):
    act = round(act,3)
    act = 0.034
    act_max = 0.07
    act_min = 0.006
    _tempo = 100 + (((act - act_min) * (420-100)) / (act_max - act_min))
    return round(_tempo)
print(selector_tempo(act))

240


Octavas

In [125]:
#Selección de Octavas Para las 2 melodías
def selector_octavas(pal_alegr,pal_trist,e1,e2,pal_total):
    js_max = 0.0375  
    js_min = -0.0009 
    js = (pal_alegr - pal_trist) / pal_total

    _octava  = 4 + round((js - js_min)*(6-4)/(js_max - js_min))

    if e1 == "joy" or e1 == "trust":
        _octava_m1 =  _octava + 1
    elif e1 == "anger" or e1 == "fear" or e1 == "sadness" or e1 == "disgust":
        _octava_m1 =  _octava - 1

    else:
        _octava_m1 = _octava

    if e2 == "joy" or e2 == "trust":
        _octava_m2 =  _octava + 1
    elif e2 == "anger" or e2 == "fear" or e2 == "sadness" or e2 == "disgust":
        _octava_m2 =  _octava - 1

    else:
        _octava_m2 = _octava 

    return _octava_m1,_octava_m2

Pitch Countour

In [126]:
def pitch_countour(positives,negatives):
    ratio = positives / negatives
    if ratio > 1:
        return 0
    else:
        return 1

Escala 

In [127]:
Cmaj_key =     ["C","D","E","F","G","A","B"] 
Cmin_key =     ["C","D","Eb","F","G","Ab","Bb"] 

def selector_escala(e1):
    escala = []
    progresion = 0
    notas = ""
    if e1 == "joy":
        escala = "Cmaj"
        notas = Cmaj_key
        progresion = 0 
    elif e1 == "sadness":
        escala = "Cmin"
        notas = Cmin_key
        progresion = 1
    elif e1 == "anger":
        escala = "Cmindis"
        notas = Cmin_key
        progresion = 1
    elif e1 == "disgust":
        escala = "Cmindis"
        notas = Cmin_key
        progresion = 1
    elif e1 == "fear":
        escala = "Cmindis7"
        notas = Cmin_key
        progresion = 1
    elif e1 == "surprise":
        escala = "Cmindis7"
        notas = Cmin_key
        progresion = 1
    return escala,progresion,notas

Progresión de Acordes

In [128]:
#major = ["I IV V I","iii vi ii V I","I V IV","I vi IV V","I V vi IV"]
#minor = ["i VI III VII","I VII VI VII","i iv v i","VI VII i i","ii v i"]

major = [[1,4,5,1],[3,6,2,5,1],[1,5,4],[1,6,4,5],[1,5,6,4]]
minor = [[1,6,3,7],[1,7,6,7],[1,4,5,1],[6,7,1,1],[2,5,1]]

def progresion(escala,progresion):
    if progresion  == 0: 
        return major[0] #random
    else:
        return minor[0]

Construyendo Progresion de Acordes

In [129]:
def acordes(escala,_progresion,notas):
    prog = progresion(escala,_progresion)
    values = []
    chords = []
    for chord in prog:
        values.append(notas[(chord - 1) % 7])     #Nota base
        values.append(notas[(chord - 1 + 2) % 7]) #Su tercera
        if escala == "Cmindis":
            values.append(notas[(chord - 1 + 4) % 7] + 'b')
        if escala == "Cmindis7":
            values.append(notas[(chord - 1 + 4) % 7] + 'b')
            values.append(notas[(chord - 1 + 5) % 7])
        else:
            values.append(notas[(chord - 1 + 4) % 7]) #Su quinta
        chords.append(values)
        values = []
    return chords

Módulo del Ritmo

In [130]:
duracion_1 = [1]   
duracion_2 = [0.5,0.5]  
duracion_3 = [0.25,0.25,0.25,0.25]
duracion_4 = [0.125,0.125,0.25,0.5]
duracion_5 = [0.0625,0.0625,0.0625,0.0625,0.25,0.5]

time_signature = 4

def ritmo(densidades):
    num_secciones = len(densidades)
    measure = []
    patron_ritmo = []
    for i in range (0,num_secciones):
        if densidades[i] == 1:
            measure = (duracion_1)
        elif densidades[i] == 2:
            measure = (duracion_2)
        elif densidades[i] == 3:
             measure = (duracion_3)
        elif densidades[i] == 4:
             measure = (duracion_3)
        elif densidades[i] == 5:
             measure = (duracion_3)
        patron_ritmo.append(measure)
        assert (np.sum(measure) == 1)
        measure = []
    return patron_ritmo

Módulo Melodía :Ponemos notas a al patrón de ritmo

In [131]:
def melodia(patron_ritmo,acordes,pitch_contour,escala):
    pitch = []
    patron_melodia = []
    it = 0
    r = 0
    for measure in patron_ritmo:
        pitch.append(acordes[it][0])       
        if len(measure) > 1:
            for nota in range (1,len(measure)):
                if measure[nota] > 0.125: #change
                     pitch.append(acordes[it][0]) #random
                else:
                    if pitch_contour == 0: 
                        pitch.append(escala[(escala.index(pitch[nota-1]) + 1) % 7])
                    else: 
                        pitch.append(escala[(escala.index(pitch[nota-1]) - 1) % 7])
        it += 1
        patron_melodia.append(pitch)
        pitch = []
    return  patron_melodia

Split Densidades

In [132]:
def split_densities(densidad):
    split = []
    desplazamiento = (max(densidad) - min(densidad)) / 5
    for seccion_densidad in densidad: 
        flag = 0
        tmp1 =  min(densidad)
        tmp2 =  min(densidad) + desplazamiento
        if flag != 1 and seccion_densidad >= tmp1 and seccion_densidad < tmp2:
            split.append(1) 
            flag = 1
        else:
            tmp1 = tmp2 + desplazamiento
        if flag != 1 and seccion_densidad >= tmp2 and seccion_densidad < tmp1:
            split.append(2)
            flag = 1
        else:
            tmp2 = tmp1 + desplazamiento
        if flag != 1 and seccion_densidad >= tmp1 and seccion_densidad < tmp2:
            split.append(3)
            flag = 1
        else:
            tmp1 = tmp2 + desplazamiento
        if flag != 1 and seccion_densidad >= tmp2 and seccion_densidad < tmp1:
            split.append(4)
            flag = 1
        else:
            tmp2 = tmp1 + desplazamiento
        if flag != 1 and seccion_densidad >= tmp1 and seccion_densidad <= max(densidad): #No 100 el des encaja
            split.append(5)
            flag = 1
    return split

Unimos Todo: Mano izquierda: Acordes y Mano derecha: Melodía

In [133]:
def modelo(patron_melodia,patron_ritmo,escala,octava,tempo,voz,acordes):
    output = ""
    measure = ""
    bar = ""
    chords = ""
    c = ""
    output += escala + "  "
    output += "V"+str(voz) + " "
    output += "T" + str(tempo) + " "
    
    #Mano izquiera
    for triada in acordes:
        for nota in triada:
            c += nota + "/1" + '+'
        c = c[:-1]
        chords += c + "  "
        c = ""
    #Mano Derecha
    for melodia,ritmo in zip(patron_melodia,patron_ritmo):
        for nota,tiempo in zip(melodia,ritmo):
            measure += nota + str(octava)+"/" + str(tiempo) + "   " 
        bar += str(measure) + str(measure) 
        measure = ""

    right_hand = output + bar
    left_hand = output + chords
    print(left_hand) 
    print(right_hand) 
    return right_hand,left_hand

Obtención de emociones del Texto

In [134]:
anger =  679
fear =  1016
surprise =  445
sadness =  1057
joy =  554
disgust =  1109


positives =  1341
negatives =  2493
pal_total =  27598


Emociones Predominates en el Texto

In [135]:
e1 = "joy"
e2 = "fear"

Calculando valores adicionales del Libro

In [136]:
activas = (joy/pal_total + anger/pal_total) 
pasivas = sadness/pal_total
act = (activas - pasivas)

Inicialización de Valores

In [138]:
tempo = selector_tempo(act)
o1,o2 = selector_octavas(joy,sadness,e1,e2,pal_total)
pitch = pitch_countour(positives,negatives)
escala,prog,notas = selector_escala(e1)
progr = progresion(escala,progresion)
chords = acordes(escala,progr,notas)
print("Tempo: ",tempo)
print("Octavas: ",o1)
print("Escala: ",escala)

Tempo:  40
Octavas:  4
Escala:  Cmaj


Evaluando Densidades

In [16]:
de1 =  [0.049, 0.051, 0.047, 0.058]
de2 =  [0.047, 0.049, 0.04, 0.048]

densidad1 = split_densities(de1)
patron_ritmo = ritmo(densidad1)
patron_melodia = melodia(patron_ritmo,chords,pitch,notas)

Generación de Melodía (e1) , (e2) , acordes

In [17]:
right_hand,left_hand = modelo(patron_melodia,patron_ritmo,escala,o1,tempo,0,chords)

Cmindis7  V0 T123 C/1+Eb/1+Gb/1+Ab/1  Ab/1+C/1+Ebb/1+F/1  Eb/1+G/1+Bbb/1+C/1  Bb/1+D/1+Fb/1+G/1  
Cmindis7  V0 T123 C3/1   C3/1   Ab3/0.5   Ab3/0.5   Ab3/0.5   Ab3/0.5   Eb3/1   Eb3/1   Bb3/0.25   Bb3/0.25   Bb3/0.25   Bb3/0.25   Bb3/0.25   Bb3/0.25   Bb3/0.25   Bb3/0.25   


Melodía (e2)

Densidades

In [ ]:
Arreglar distribucion . Analizar densidades limpiando lematizando todito :)    
Si termina en 8 0 16 . aumentar un measure mas con 1 o un acorde 
Limpia el texto y si densidadades con respecto a num plab o a la cant de emociones OK.
cambiar valores del tempo y octavas por los nuestros :)
Se divide el texto en secciones.
De cada seccion calcular . Cuántas palabras hay con e1 e2
Tendrás un array [20 10 40 50 ... 80]
Si el número es > entonces, más densidad de notas, porque hay más de esa emoción en dicha seccio
ver ideas cel